In [1]:
using DataFrames, CSV, StringEncodings, Dictionaries, Dates, StatsBase, HTTP, JSON, ProgressBars

In [31]:
yr = 2023
datadir = joinpath("..", "Data", string(yr))

"../Data/2023"

In [36]:
# Here we prepare the raw SP data frames to merge them correctly

spnames = DataFrame(CSV.File(joinpath(datadir, "SP_names.csv")))
spteams = DataFrame(CSV.File(joinpath(datadir, "SP_teams.csv")))
datadf = DataFrame(CSV.File(joinpath(datadir, "pitching_data_with_results.csv")))

spnames.join_key = string.(spnames.game_pk) .* string.(spnames.pitches) .* string.(spnames.babip) .* string.(spnames.launch_speed) .* string.(spnames.launch_angle)
spteams.join_key = string.(spteams.game_pk) .* string.(spteams.pitches) .* string.(spteams.babip) .* string.(spteams.launch_speed) .* string.(spteams.launch_angle)

a = size(spnames, 1)
b = size(spteams, 1)

println("Height of each DF: names = $a, teams = $b")

spnames = spnames[!, [:join_key, :game_pk, :player_id, :player_name]]
spteams = spteams[!, [:join_key, :player_id, :player_name]]
rename!(spteams, [:player_id, :player_name] .=> [:team_id, :team_name])

spdf = innerjoin(spnames, spteams, on=:join_key)
c = size(spdf, 1)

println("Height of joined df: $c")

if a == b && b == c
    spdf.join_key = string.(spdf.game_pk) .* string.(spdf.team_id)
    rename!(spdf, [:player_id, :player_name] .=> [:spitcher_id, :spitcher_name])
    spdf = spdf[!, [:join_key, :spitcher_id, :spitcher_name]]
    
    datadf.join_key = string.(datadf.game_pk) .* string.(datadf.player_id)
    leftjoin!(datadf, spdf, on=:join_key)
end

Height of each DF: names = 4860, teams = 4860
Height of joined df: 4860


Row,pitches,player_id,player_name,game_date,game_pk,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance,pitcher_run_exp,run_exp,bat_speed,swing_length,pa,bip,singles,doubles,triples,hrs,so,k_percent,bb,bb_percent,api_break_z_with_gravity,api_break_z_induced,api_break_x_arm,api_break_x_batter_in,hyper_speed,bbdist,hardhit_percent,barrels_per_bbe_percent,barrels_per_pa_percent,release_pos_z,release_pos_x,plate_x,plate_z,obp,barrels_total,batter_run_value_per_100,xobp,xslg,pitcher_run_value_per_100,xbadiff,xobpdiff,xslgdiff,wobadiff,swing_miss_percent,arm_angle,attack_angle,attack_direction,swing_path_tilt,rate_ideal_attack_angle,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,HomeID,AwayID,HomeRuns,AwayRuns,join_key,spitcher_id,spitcher_name
,Int64,Int64,String3,Date,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Int64?,Float64,Float64?,Int64,Int64,Int64,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64,Float64,Float64?,Float64?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64,Int64,Int64,Int64,String,Int64?,String31?
1,245,133,OAK,2023-04-14,718578,246,99.6,0.297,0.189,0.345,0.486,0.456,0.426,0.247,11,37,89.4,15.5,2041,87.1,86.52,20,82,163,-0.6,5.88,112,144,116,145,289,317,298,-5.561,5.561,missing,missing,56,30,6,4,0,1,7,12.5,17,30.4,2.32813,0.85545,0.67177,-0.16453,92.1,166,46.6667,6.66667,3.57143,5.93,-1.51,-0.06,2.15,0.536,2,2.2698,0.502,0.426,-2.2698,0.05,0.034,0.06,0.03,24.4,36.7,missing,missing,missing,missing,missing,missing,133,121,6,17,718578133,621076,"Kaprielian, James"
2,240,115,COL,2023-06-06,717869,240,100.0,0.359,0.256,0.583,0.615,0.468,0.416,0.252,14,39,94.1,15.9,2407,90.5,90.52,19,93,147,0.0,5.98,113,147,117,147,311,330,300,-5.801,5.801,missing,missing,52,24,5,8,1,0,16,30.8,11,21.2,2.28608,0.49261,0.25059,-0.13812,91.7,176,62.5,25.0,11.5385,6.0,-1.43,-0.04,2.59,0.5,6,2.41708,0.432,0.51,-2.41708,0.107,0.068,0.105,0.052,20.4,40.4,missing,missing,missing,missing,missing,missing,115,137,4,10,717869115,659275,"Lamet, Dinelson"
3,240,140,TEX,2023-08-27,716824,240,100.0,0.182,0.068,0.233,0.25,0.28,0.318,0.215,8,44,89.3,29.8,2305,91.0,91.67,34,102,138,0.7,6.56,113,150,116,146,298,321,292,2.666,-2.666,missing,missing,54,31,7,0,0,1,13,24.1,10,18.5,1.99892,0.777,0.42529,-0.38196,92.0,179,38.7097,3.22581,1.85185,6.27,0.21,0.23,2.3,0.333,1,-1.11083,0.36,0.329,1.11083,-0.033,-0.027,-0.079,-0.038,33.3,42.9,missing,missing,missing,missing,missing,missing,142,140,7,6,716824140,656756,"Montgomery, Jordan"
4,238,146,MIA,2023-08-02,717159,239,99.6,0.255,0.149,0.344,0.404,0.334,0.322,0.254,12,47,88.3,4.2,2298,90.3,90.46,34,104,134,0.2,6.54,105,145,114,146,298,315,295,0.245,-0.245,68.4,7.2,54,33,7,4,0,1,14,25.9,6,11.1,2.19452,0.64592,0.42929,0.15498,91.5,126,39.3939,3.0303,1.85185,5.71,0.77,-0.31,2.18,0.352,1,-0.102941,0.352,0.359,0.102941,0.001,0.0,0.045,0.012,32.7,38.3,9.7887,-3.07954,31.2708,0.529412,35.1918,29.535,146,143,9,8,717159146,666129,"Garrett, Braxton"
5,236,117,HOU,2023-08-24,716873,236,100.0,0.51,0.306,0.55,0.816,0.573,0.425,0.337,25,49,94.2,6.3,2180,82.9,81.79,16,100,136,-1.1,5.7,113,149,117,149,285,324,290,-10.882,10.882,71.0,7.4,58,43,16,6,0,3,7,12.1,6,10.3,3.27699,0.72208,0.40729,-0.13127,93.9,161,53.4884,13.9535,10.3448,6.04,-1.17,0.0,2.43,0.569,6,4.61102,0.423,0.583,-4.61102,0.173,0.146,0.233,0.148,16.0,45.6,14.0923,-3.18927,31.1539,0.547368

In [37]:
# Add on winning team

HomeTeamWin = datadf.HomeRuns .> datadf.AwayRuns
IsHomeTeam = datadf.player_id .== datadf.HomeID

datadf.winner = .!(xor.(HomeTeamWin, IsHomeTeam))

sum(ismissing.(datadf.spitcher_id))

0

In [38]:
CSV.write(joinpath(datadir, "PitchingDataWithResultsAndStarters.csv"), datadf)

"../Data/2023/PitchingDataWithResultsAndStarters.csv"

In [51]:
homedf = datadf[datadf.player_id .== datadf.HomeID, :]
sort!(homedf, [:game_pk])
awaydf = datadf[datadf.player_id .== datadf.AwayID, :]
sort!(awaydf, [:game_pk])

sort!(datadf, [:game_pk])

datadf.offensive_xwoba = NaN .* zeros(size(datadf, 1))
datadf.offensive_launch_angle = NaN .* zeros(size(datadf, 1))
datadf.offensive_launch_speed = NaN .* zeros(size(datadf, 1))
datadf.offensive_hardhit_percent = NaN .* zeros(size(datadf, 1))

datadf[datadf.player_id .== datadf.HomeID, [:offensive_xwoba, 
                        :offensive_launch_angle, :offensive_launch_speed, 
                                                :offensive_hardhit_percent]] = Array(awaydf[!, [:xwoba, :launch_angle,
                                                                                    :launch_speed, :hardhit_percent]]);

datadf[datadf.player_id .== datadf.AwayID, [:offensive_xwoba, 
                        :offensive_launch_angle, :offensive_launch_speed, 
                                                :offensive_hardhit_percent]] = Array(homedf[!, [:xwoba, :launch_angle,
                                                                                    :launch_speed, :hardhit_percent]]);


In [74]:
# Here let's add on a last five game offense / last game with pitcher 

last5_o_xwoba = NaN .* ones(size(datadf, 1))
last5_o_launch_angle = NaN .* ones(size(datadf, 1))
last5_o_launch_speed = NaN .* ones(size(datadf, 1))
last5_o_hardhit_percent = NaN .* ones(size(datadf, 1))

last_p_xwoba = NaN .* ones(size(datadf, 1))
last_p_launch_angle = NaN .* ones(size(datadf, 1))
last_p_launch_speed = NaN .* ones(size(datadf, 1))
last_p_hardhit_percent = NaN .* ones(size(datadf, 1))

idx = 1

for r = ProgressBar(eachrow(datadf))
    last5df = datadf[(datadf.game_date .<= r.game_date) .& 
                    (datadf.player_id .== r.player_id) .&
                    (datadf.game_pk .!= r.game_pk), :]
    sort!(last5df, :game_date)
    
    last5df = last(last5df, 5)

    last5_o_hardhit_percent[idx] = mean(last5df.offensive_hardhit_percent)
    last5_o_launch_angle[idx] = mean(last5df.offensive_launch_angle)
    last5_o_launch_speed[idx] = mean(last5df.offensive_launch_speed)
    last5_o_xwoba[idx] = mean(last5df.offensive_xwoba)

    lastpitchdf = datadf[(datadf.game_date .<= r.game_date) .& 
                    (datadf.player_id .== r.player_id) .&
                    (datadf.game_pk .!= r.game_pk) .& 
                    (datadf.spitcher_id .== r.spitcher_id), :]

    sort!(last5df, :game_date)
    last5df = last(last5df, 1)     
    
    last_p_hardhit_percent[idx] = mean(lastpitchdf.hardhit_percent)
    last_p_launch_angle[idx] = mean(lastpitchdf.launch_angle)
    last_p_launch_speed[idx] = mean(lastpitchdf.launch_speed)
    last_p_xwoba[idx] = mean(lastpitchdf.xwoba)

    idx = idx + 1
end

datadf.last5_o_hardhit_percent = last5_o_hardhit_percent
datadf.last5_o_launch_angle = last5_o_launch_angle
datadf.last5_o_launch_speed = last5_o_launch_speed
datadf.last5_o_xwoba = last5_o_xwoba

datadf.last_p_hardhit_percent = last_p_hardhit_percent
datadf.last_p_launch_angle = last_p_launch_angle
datadf.last_p_launch_speed = last_p_launch_speed
datadf.last_p_xwoba = last_p_xwoba



0.0%┣                                             ┫ 0/4.9k [00:00<00:00, -0s/it]
7.5%┣███                                      ┫ 366/4.9k [00:00<00:01, 7.2kit/s]
17.0%┣██████▉                                 ┫ 824/4.9k [00:00<00:00, 8.2kit/s]
26.5%┣██████████▍                            ┫ 1.3k/4.9k [00:00<00:00, 8.5kit/s]
36.4%┣██████████████▏                        ┫ 1.8k/4.9k [00:00<00:00, 8.8kit/s]
46.7%┣██████████████████▏                    ┫ 2.3k/4.9k [00:00<00:00, 9.0kit/s]
57.4%┣██████████████████████▍                ┫ 2.8k/4.9k [00:00<00:00, 9.2kit/s]
68.1%┣██████████████████████████▋            ┫ 3.3k/4.9k [00:00<00:00, 9.4kit/s]
79.5%┣███████████████████████████████        ┫ 3.9k/4.9k [00:00<00:00, 9.6kit/s]
90.6%┣███████████████████████████████████▍   ┫ 4.4k/4.9k [00:00<00:00, 9.7kit/s]
100.0%┣██████████████████████████████████████┫ 4.9k/4.9k [00:00<00:00, 9.9kit/s]
100.0%┣██████████████████████████████████████┫ 4.9k/4.9k [00:00<00:00, 9.9kit/s]


4860-element Vector{Float64}:
   0.285
   0.3378461538461538
   0.30023809523809525
   0.331
   0.32
   0.3615
   0.29933333333333334
   0.3113333333333333
   0.3375
   0.32080000000000003
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [78]:
datadf

Row,pitches,player_id,player_name,game_date,game_pk,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance,pitcher_run_exp,run_exp,bat_speed,swing_length,pa,bip,singles,doubles,triples,hrs,so,k_percent,bb,bb_percent,api_break_z_with_gravity,api_break_z_induced,api_break_x_arm,api_break_x_batter_in,hyper_speed,bbdist,hardhit_percent,barrels_per_bbe_percent,barrels_per_pa_percent,release_pos_z,release_pos_x,plate_x,plate_z,obp,barrels_total,batter_run_value_per_100,xobp,xslg,pitcher_run_value_per_100,xbadiff,xobpdiff,xslgdiff,wobadiff,swing_miss_percent,arm_angle,attack_angle,attack_direction,swing_path_tilt,rate_ideal_attack_angle,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,HomeID,AwayID,HomeRuns,AwayRuns,join_key,spitcher_id,spitcher_name,winner,offensive_xwoba,offensive_launch_angle,offensive_launch_speed,offensive_hardhit_percent,last5_o_hardhit_percent,last5_o_launch_angle,last5_o_launch_speed,last5_o_xwoba,last_p_hardhit_percent,last_p_launch_angle,last_p_launch_speed,last_p_xwoba
,Int64,Int64,String3,Date,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Int64?,Float64,Float64?,Int64,Int64,Int64,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64,Float64,Float64?,Float64?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64,Int64,Int64,Int64,String,Int64?,String31?,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,133,147,NYY,2023-10-01,716352,133,100.0,0.344,0.344,0.364,0.688,0.425,0.386,0.327,11,32,89.7,0.8,2192,90.5,90.64,17,68,65,0.1,6.14,108,149,108,150,304,337,302,-3.594,3.594,68.8,7.1,35,26,7,0,1,3,8,22.9,1,2.9,2.20594,0.58744,0.51955,-0.10165,91.9,170,46.1538,11.5385,8.57143,5.73,-2.16,0.04,2.44,0.353,3,2.70226,0.339,0.588,-2.70226,0.017,0.014,0.1,0.039,25.0,36.8,8.13597,3.57495,31.8551,0.42623,38.9723,27.0882,118,147,5,2,716352147,650633,"King, Michael",false,0.356,8.3,92.0,52.0,41.5154,9.12,88.36,0.3128,37.6081,13.4125,88.075,0.285
2,130,118,KC,2023-10-01,716352,130,100.0,0.219,0.094,0.28,0.313,0.267,0.356,0.294,7,32,92.0,8.3,2248,87.2,87.41,18,64,66,0.2,6.18,113,150,123,148,300,325,292,1.305,-1.305,69.0,7.4,35,25,4,3,0,0,7,20.0,3,8.6,2.72008,0.32215,0.16931,-0.02623,92.8,147,52.0,8.0,5.71429,6.05,-1.54,0.08,2.23,0.286,2,-1.00385,0.355,0.469,1.00385,-0.075,-0.069,-0.156,-0.089,28.1,41.8,9.20645,0.431112,32.0585,0.566667,38.7732,28.9211,118,147,5,2,716352118,425844,"Greinke, Zack",true,0.386,0.8,89.7,46.1538,36.2455,12.28,85.78,0.2978,39.4869,13.2654,88.5154,0.337846
3,132,113,CIN,2023-10-01,716353,132,100.0,0.3,0.1,0.409,0.4,0.329,0.343,0.272,9,30,89.8,15.0,2344,91.6,91.55,13,65,67,0.0,6.32,107,149,122,149,301,327,308,0.283,-0.283,66.9,7.0,35,23,6,3,0,0,9,25.7,1,2.9,2.21197,0.55015,0.23409,-0.10515,91.9,161,47.8261,13.0435,8.57143,5.74,-2.09,0.01,2.44,0.353,3,-0.214394,0.329,0.478,0.214394,0.028,0.024,-0.078,-0.014,20.0,28.0,6.18023,-3.81157,29.5004,0.459016,33.8359,27.9112,138,113,4,3,716353113,668881,"Greene, Hunter",false,0.343,25.5,86.5,42.8571,44.7293,13.04,89.3,0.3706,38.4338,15.5429,87.7143,0.300238
4,118,138,STL,2023-10-01,716353,118,100.0,0.257,0.2,0.4,0.457,0.3,0.343,0.252,9,35,86.5,25.5,2273,88.3,88.4,26,72,46,0.1,6.38,111,148,123,148,307,322,307,1.179,-1.179,69.2,7.1,35,21,4,4,0,1,14,40.0,0,0.0,2.57797,0.42578,0.1978,-0.17017,91.2,187,42.8571,23.8095,14.2857,6.4,-1.65,0.29,2.36,0.257,5